In [29]:

import argparse
import http.client
import httplib2
import os
import random
import time
import datetime
from datetime import datetime, timedelta
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
from DashboardFunctions import *
import os
from dotenv import load_dotenv
from sqlalchemy import *
import datetime
from Google import Create_Service
import re
from IPython.display import Image
load_dotenv()
from StreamMaster import *
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from twitchAPI.twitch import Twitch
from copy import copy, deepcopy
from jupyter_dash import JupyterDash
import dash
from dash import dcc, Dash
from dash import html
from matplotlib.colors import *
import random

ModuleNotFoundError: No module named 'matplotlib'

In [69]:
with Session(mysql_engine) as session:
    # session.query(Clip_Tracker).where(Clip_Tracker.title.ilike('%Meow%pt%')).delete(synchronize_session='fetch')
    print(session.query(Clip_Tracker).where(Clip_Tracker.title.ilike('%Meow%pt%')).all())
    # session.commit()

[]


In [34]:
with Session(mysql_engine) as session:
        print(session.query(Clip_Tracker.title, Clip_Tracker.mobiles_videos_processed).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())

[('Work smart and not hard?', False), ('Just in Time Revive', True), ('Speak sense soldier', False), ('This Thing has no Range!!', False), ('You blind!', False), ('How do you run?', False), ('SquareWon is a Multitasker', False), ('Who is controlling the Signs', False), ('Meow Full Gameplay pt2', False), ('How to survive the Sewers', False), ('Killing our nemesis (Harrow)', False), ('Just chaos', False), ('Mini Boss battle progress!', True), ('SquareWon takeover: dropping the racoon', True), ('phrasing', True), ('The next 4 don’t need chocolate', True), ('Set on fire', True), ('The Mortar is not a defense tool', True), ('Is she hiding?', True), ('The best weapons for Raze (star boss)', True), ('Always save the Princess', True), ('Remember to kill all the skulls when fighting Raze!', True), ('Fireball magic', True), ('Co-Op: We can do it', True), ('Tough boss', True), ('Why we love Remnant Survivor', True), ('I don’t wanna die', True), ('Elixir management', True), ('Did we win there?', T

In [ ]:
with Session(mysql_engine) as session:
    # session.execute(delete(Clip_Tracker ).where(Clip_Tracker.title.ilike('%RoundWon rating%')), execution_options={"synchronize_session": 'fetch'})
    # print(session.query(Clip_Tracker).where(Clip_Tracker.title.ilike('%RoundWon rating%')).all())
    print(session.query(Game_Meta.game_name).where(Game_Meta.game_name.ilike("%Call of %")).all())
    # session.commit()

In [ ]:
mysql_engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER_NAME']}:{os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" )


clips_folder = os.environ['CLIPS_FOLDER']
mobile_folder = os.environ['MOBILE_FOLDER']
fullscreen_folder = os.environ['FULLSCREEN_FOLDER']
ml_folder = os.environ['ML_FOLDER']
assets = os.environ['ASSETS']

CLIENT_SECRET_FILE = 'client_secrets.json'
# YT_SECRET_FILE = 'client_secrets_yt.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
# recommend pulling as much access as possible since the Create_Service function will create a pickle token of your authentication allow single authentication for extended period.
SCOPES = ["https://www.googleapis.com/auth/youtube.upload", 'https://www.googleapis.com/auth/youtube']

# Code that initiates connection to the Google API - pulled from Google.py. Difference from using Google's built-in service is the following script creates convenience of storing pickle with necessary authentication
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

In [ ]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

In [ ]:
twitch = Twitch(os.environ['TWITCH_APP_ID'], os.environ['TWITCH_APP_SECRET'])

In [ ]:
color_sample = px.colors.sequential.YlOrBr + px.colors.sequential.ice
random.seed(333)
random.shuffle(color_sample)
game_colors = {}
games = {}
cnt = 0
with Session(mysql_engine) as session:
    for game_id, game_name in session.query(Game_Meta.game_id,func.max(Game_Meta.game_name)).select_from(join(Game_Meta,Clip_Tracker)).group_by(Game_Meta.game_id).all():
        games[game_id] = game_name 
        game_colors[game_name] = color_sample[cnt]
        cnt += 1

In [ ]:
color_sample = px.colors.sequential.Viridis

with Session(mysql_engine) as session:
    creators = {creator[0]: color_sample[i] for i, creator in enumerate(session.query(distinct(Clip_Tracker.creator_name)).all())}

In [ ]:
# Build sa visual for un-published clips
data_seed = {}
with Session(mysql_engine) as session:
    # for url in session.query(Clip_Tracker.url).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all()[:2]:
    for title, game_name, created_date, view_count in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.created_at,  Clip_Tracker.view_count).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all():
        if bool(data_seed.get(game_name)):
            try:
                data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
                data_seed[game_name]['y'].append(view_count)
                data_seed[game_name]['text'].append(title)
            except:
                data_seed[game_name]['x'] = [created_date]
                data_seed[game_name]['y'] = [view_count]
                data_seed[game_name]['color'] = game_colors[game_name]
        else:
            data_seed[game_name] = {'x': [], 'y': [], 'text': [] }
            data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
            data_seed[game_name]['y'].append(view_count)
            data_seed[game_name]['text'].append(title)
            data_seed[game_name]['color'] = game_colors[game_name]
    # print(session.query(distinct(Clip_Tracker.game_id), Game_Meta.game_name).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())
        # print(url[0])

data = []
for key, value in data_seed.items():
    data.append(go.Bar(x=value['x'], y = value['y'], text = value['text'], name=key, marker={'color': value['color']}))
go.Figure(data, go.Layout(barmode='stack', title={'text': "Performance of Unpublished Clips"}))

In [ ]:
# Build sa visual for Published clips
data_seed = {}
with Session(mysql_engine) as session:
    # for url in session.query(Clip_Tracker.url).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all()[:2]:
    for title, game_name, created_date, view_count in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.created_at,  Clip_Tracker.view_count).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published != None).order_by(Clip_Tracker.view_count.desc()).all():
        if bool(data_seed.get(game_name)):
            try:
                data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
                data_seed[game_name]['y'].append(view_count)
                data_seed[game_name]['text'].append(title)
            except:
                data_seed[game_name]['x'] = [created_date]
                data_seed[game_name]['y'] = [view_count]
        else:
            data_seed[game_name] = {'x': [], 'y': [], 'text': []}
            data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
            data_seed[game_name]['y'].append(view_count)
            data_seed[game_name]['text'].append(title)
            data_seed[game_name]['color'] = game_colors[game_name]

data = []
for key, value in data_seed.items():
    data.append(go.Bar(x=value['x'], y = value['y'], text = value['text'], name=key,  marker={'color': value['color']}))
go.Figure(data, go.Layout(barmode='stack', title={'text': "Performance of Published Clips"}))
    # print(session.query(distinct(Clip_Tracker.game_id), Game_Meta.game_name).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())
        # print(url[0])



In [ ]:
# Build sa visual for un-published clips
data_seed = {}
category_mapper = {'m': "Mobile Only", 'f': "Fully Published", "d": 'Desktop Only'}
with Session(mysql_engine) as session:
    # for url in session.query(Clip_Tracker.url).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all()[:2]:
    for title, game_name, view_count, publishing_status, created_date in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.view_count, Clip_Tracker.published, Clip_Tracker.created_at).select_from(join(Clip_Tracker, Game_Meta)).order_by(Clip_Tracker.view_count.desc()).all():
        if isinstance(publishing_status, PublishingStatus):
            publshing_status_name = category_mapper[publishing_status.name]
        else:
            if publishing_status != None:
                print(publishing_status)
            publshing_status_name = "Not Published"
        if bool(data_seed.get(publshing_status_name)):
            data_seed[publshing_status_name]['y'].append(game_name)
            data_seed[publshing_status_name]['x'].append(view_count)
            data_seed[publshing_status_name]['text'].append(title)
        else:
            data_seed[publshing_status_name] = {'x': [], 'y': [], 'text': []}
            data_seed[publshing_status_name]['x'].append(view_count)
            data_seed[publshing_status_name]['y'].append(game_name)
            data_seed[publshing_status_name]['text'].append(title)


    # print(session.query(distinct(Clip_Tracker.game_id), Game_Meta.game_name).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())
        # print(url[0])

data = []

odered_list = ['Fully Published', "Mobile Only", 'Desktop Only', "Not Published"]
# for key, value in data_seed.items():
for key in odered_list:
    category_name = key
    data.append(go.Bar(x=data_seed[key]['x'], y = data_seed[key]['y'], text = data_seed[key]['text'], name= category_name, orientation='h'))
go.Figure(data, go.Layout(barmode='stack', title={'text': "Performance By Game and Publishing Status"}, xaxis = {'title': {'text': "View Count"}},yaxis={'categoryorder': 'total ascending', 'title': {'text': 'Game Title'}}))

In [ ]:
data = []

fig = make_subplots(rows=1, cols = 4, subplot_titles = ("Average Views per Clip", "Max Views for Clip", "Total Views of Clips", "Number of Clips"), shared_yaxes=True)
with Session(mysql_engine) as session:
    # for title, game_name, view_count, publishing_status in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.view_count, Clip_Tracker.published).select_from(join(Clip_Tracker, Game_Meta)).order_by(Clip_Tracker.view_count.desc()).all():
    cnt = 0
    for game_name, avg_cnt, max_cnt, sum_cnt, video_cnt in session.query(Game_Meta.game_name, func.avg(Clip_Tracker.view_count), func.max(Clip_Tracker.view_count), func.sum(Clip_Tracker.view_count), func.count(Clip_Tracker.title)).select_from(join(Game_Meta, Clip_Tracker)).group_by(Clip_Tracker.game_id).order_by(func.sum(Clip_Tracker.view_count).desc()).all():
        fig.add_trace(go.Bar(x=[avg_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')),row=1, col=1)
        fig.add_trace(go.Bar(x=[max_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')),row = 1, col=2)
        fig.add_trace(go.Bar(x=[sum_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')), row = 1, col=3)
        fig.add_trace(go.Bar(x=[video_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')), row=1,  col=4)
        # # data.append(go.Bar(x=[avg_cnt], y = [game_name]))
        cnt +=1
fig.update_yaxes(type='category')
fig.update_layout(barmode='group', yaxis={'categoryorder': 'category descending'}, title={'text': "Breaking each game by different statistics"})
fig.show()
        

In [ ]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None

In [19]:
SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'client_secrets_desktop.json'

def get_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()

  return response
youtubeAnalytics = Create_Service(CLIENT_SECRETS_FILE, API_SERVICE_NAME, API_VERSION, SCOPES)
# if __name__ == '__main__':
#   # Disable OAuthlib's HTTPs verification when running locally.
#   # *DO NOT* leave this option enabled when running in production.
#   os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

# youtubeAnalytics = get_service()

client_secrets_desktop.json-youtubeAnalytics-v2-(['https://www.googleapis.com/auth/yt-analytics.readonly'],)
['https://www.googleapis.com/auth/yt-analytics.readonly']
youtubeAnalytics service created successfully


In [21]:
video_metrics

{'kind': 'youtubeAnalytics#resultTable',
 'columnHeaders': [{'name': 'video',
   'columnType': 'DIMENSION',
   'dataType': 'STRING'},
  {'name': 'estimatedMinutesWatched',
   'columnType': 'METRIC',
   'dataType': 'INTEGER'},
  {'name': 'views', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'likes', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'dislikes', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'subscribersGained', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'subscribersLost', 'columnType': 'METRIC', 'dataType': 'INTEGER'}],
 'rows': [['f0E8gLxw-II', 306, 511, 6, 1, 0, 0],
  ['fuIgHicjm0k', 233, 871, 34, 2, 3, 0],
  ['1dSor9KrL34', 229, 927, 8, 2, 0, 0],
  ['hFgJfliPTpA', 175, 596, 17, -1, 0, 0],
  ['NFDigvjBh2A', 165, 782, 16, 5, 0, 0],
  ['Q9ipoByqDHk', 137, 474, 12, 3, 4, 0],
  ['zhHWS92VrYY', 124, 260, 9, 1, 0, 0],
  ['xKtv8kaYyXI', 74, 331, 9, 1, 0, 0],
  ['FvT6puEvVKw', 73, 222, 1, 0, 0, 0],
  ['6ZcTqh_6TLk', 62, 250, 3

In [20]:
video_metrics = execute_api_request(
      youtubeAnalytics.reports().query,
      ids='channel==MINE',
      startDate='2022-03-01',
      endDate='2022-07-12',
      maxResults = 140,
      metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
      dimensions='video',
      sort='-estimatedMinutesWatched'
  )

In [ ]:
video_seed = {'video_id': [], 'style': [],'game_name': [], 'title': [], 'creator': [], 'view_count': []}
with Session(mysql_engine) as session:
    for video_id, style, game_name,  title, creator,  view_count in session.query(Video_yt.id, Video_yt.style, Game_Meta.game_name, Clip_Tracker.title, Clip_Tracker.creator_name, Clip_Tracker.view_count ).select_from(join(join(Clip_Tracker, Game_Meta), Video_yt)).where(Clip_Tracker.published != None).all():
        video_seed['video_id'].append(video_id)
        video_seed['style'].append(style.name)
        video_seed['game_name'].append(game_name)
        video_seed['title'].append(title)
        video_seed['creator'].append(creator)
        video_seed['view_count'].append(view_count)


In [ ]:
clip_metrics_df = pd.DataFrame(video_metrics['rows'], columns=['video_id', 'estimatedMinutesWatched', 'views', 'likes', 'dislikes', 'subscribersGained', 'subscribersLost']).set_index('video_id').join(pd.DataFrame(video_seed).set_index('video_id'))

'2022-07-27'

In [48]:
video_metrics = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-04-01',
        endDate='2022-07-27',
        maxResults = 140,
        metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
        dimensions='video',
        sort='-estimatedMinutesWatched'
    )

video_seed = {'video_id': [], 'style': [],'game_name': [], 'title': [], 'creator': [], 'view_count': []}
with Session(mysql_engine) as session:
    for video_id, style, game_name,  title, creator,  view_count in session.query(Video_yt.id, Video_yt.style, Game_Meta.game_name, Clip_Tracker.title, Clip_Tracker.creator_name, Clip_Tracker.view_count ).select_from(join(join(Clip_Tracker, Game_Meta), Video_yt)).where(Clip_Tracker.published != None).all():
        video_seed['video_id'].append(video_id)
        video_seed['style'].append(style.name)
        video_seed['game_name'].append(game_name)
        video_seed['title'].append(title)
        video_seed['creator'].append(creator)
        video_seed['view_count'].append(view_count)

In [49]:
video_metrics

{'kind': 'youtubeAnalytics#resultTable',
 'columnHeaders': [{'name': 'video',
   'columnType': 'DIMENSION',
   'dataType': 'STRING'},
  {'name': 'estimatedMinutesWatched',
   'columnType': 'METRIC',
   'dataType': 'INTEGER'},
  {'name': 'views', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'likes', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'dislikes', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'subscribersGained', 'columnType': 'METRIC', 'dataType': 'INTEGER'},
  {'name': 'subscribersLost', 'columnType': 'METRIC', 'dataType': 'INTEGER'}],
 'rows': [['f0E8gLxw-II', 308, 517, 6, 1, 0, 0],
  ['XaHu2ApQ1kM', 242, 1286, 30, 2, 1, 0],
  ['fuIgHicjm0k', 235, 881, 34, 2, 3, 0],
  ['1dSor9KrL34', 231, 938, 8, 2, 0, 0],
  ['hFgJfliPTpA', 178, 604, 17, -1, 0, 0],
  ['NFDigvjBh2A', 166, 786, 16, 5, 0, 0],
  ['Q9ipoByqDHk', 137, 476, 12, 3, 4, 0],
  ['shYDWo24xSQ', 136, 362, 11, 1, 0, 0],
  ['zhHWS92VrYY', 125, 264, 10, 1, 0, 0],
  ['4Ys54dc1ZeA', 95, 

In [50]:
# video_seed, video_metrics = build_video_seed(youtubeAnalytics)
clip_metrics_df = pd.DataFrame(video_metrics['rows'], columns=['video_id', 'estimatedMinutesWatched', 'views', 'likes', 'dislikes', 'subscribersGained', 'subscribersLost']).set_index('video_id').join(pd.DataFrame(video_seed).set_index('video_id'))
clip_metrics_df['likes_ratio'] = (clip_metrics_df['likes']/(clip_metrics_df['likes'] + clip_metrics_df['dislikes'])).fillna(0)

In [3]:
sub_group = clip_metrics_df.groupby('title').min()
game_sub = sub_group.loc[sub_group['game_name'] == "God of War"]

In [51]:
clip_metrics_df.head()

,estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost,style,game_name,title,creator,view_count,likes_ratio
video_id,,,,,,,,,,,,
f0E8gLxw-II,308,517,6,1,0,0,m,Remnant: From the Ashes,His weakness is the Butt,TheRoundWon,51.0,0.857143
XaHu2ApQ1kM,242,1286,30,2,1,0,NaN,NaN,NaN,NaN,NaN,0.937500
fuIgHicjm0k,235,881,34,2,3,0,m,Demon's Souls,Iconic,FeedMeACat,53.0,0.944444
1dSor9KrL34,231,938,8,2,0,0,m,Horizon Forbidden West,Enemies can do this now,TheRoundWon,10.0,0.800000
hFgJfliPTpA,178,604,17,-1,0,0,m,Remnant: From the Ashes,Remember to roll forward,TheRoundWon,61.0,1.062500


In [47]:
clip_metrics_df.dropna(axis=0, how= 'any', inplace=True)
clip_metrics_df.loc[clip_metrics_df.title.str.contains('Heroes')]

,estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost,style,game_name,title,creator,view_count,likes_ratio
video_id,,,,,,,,,,,,
fu4JKw_ACtI,3,9,0,0,0,0,f,Horizon Forbidden West,Heroes Don't Die,PraetorJoe,19.0,0.0
giUFF6fYIhY,1,7,0,0,0,0,m,Horizon Forbidden West,Heroes Don't Die,PraetorJoe,19.0,0.0


In [15]:
video_metrics = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-03-01',
        endDate='2022-07-12',
        maxResults = 140,
        metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
        dimensions='video',
        sort='-estimatedMinutesWatched'
    )

In [18]:
video_metrics

In [55]:
fig = make_subplots(rows=1, cols=2, subplot_titles = ("Twitch Views", "Youtube Shorts Views"), shared_yaxes=True)

for ix in clip_metrics_df.loc[(clip_metrics_df['game_name'].notnull()) & (clip_metrics_df['style'] == 'm'), :].index:
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'view_count']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'view_count']], 'coloraxis': 'coloraxis'}), row=1, col=1)
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'views']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'view_count']], 'coloraxis': 'coloraxis'}), row=1, col=2)

fig.update_layout(height = 400, barmode='stack', title={'text': "Published Shorts vs Clips "}, yaxis2 = {'categoryorder': "total ascending"} )
fig.update_coloraxes(showscale=False)
fig.show()


In [56]:
fig.layout.height

400

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles = ("Twitch Views", "Youtube Full Screen Views"), shared_yaxes=True)

for ix in clip_metrics_df.loc[(clip_metrics_df['game_name'].notnull()) & (clip_metrics_df['style'] == 'f'), :].index:
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'view_count']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'view_count']], 'coloraxis': 'coloraxis'}), row=1, col=1)
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'views']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'views']], 'coloraxis': 'coloraxis'}), row=1, col=2)

fig.update_layout(barmode='stack', title={'text': "Published Full Screens vs Clips "})
fig.update_coloraxes(showscale=False)
fig.show()


In [56]:
clip_metrics_df['likes_ratio'] = (clip_metrics_df['likes']/(clip_metrics_df['likes'] + clip_metrics_df['dislikes'])).fillna(0)

In [ ]:
game_sub = clip_metrics_df.loc[clip_metrics_df['game_name'] == "Demon's Souls"]

In [38]:
clip_metrics_df

,estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost,style,game_name,title,creator,view_count
video_id,,,,,,,,,,,
f0E8gLxw-II,306,511,6,1,0,0,m,Remnant: From the Ashes,His weakness is the Butt,TheRoundWon,51.0
fuIgHicjm0k,233,871,34,2,3,0,m,Demon's Souls,Iconic,FeedMeACat,53.0
1dSor9KrL34,229,927,8,2,0,0,m,Horizon Forbidden West,Enemies can do this now,TheRoundWon,10.0
hFgJfliPTpA,175,596,17,-1,0,0,m,Remnant: From the Ashes,Remember to roll forward,TheRoundWon,61.0
NFDigvjBh2A,165,782,16,5,0,0,m,Demon's Souls,Back to reality,TheRoundWon,10.0
...,...,...,...,...,...,...,...,...,...,...,...
49WSCEmA7X4,0,1,0,0,0,0,m,Clash Royale,My must watch players,TheRoundWon,7.0
cXdbITNtwPw,0,2,0,0,0,0,f,Horizon Forbidden West,Weird people on horses,TheRoundWon,7.0
V1FdmoOpP3Q,0,1,0,0,0,0,m,Remnant: From the Ashes,Pessimism vs Realism? (or Optimism?),TheRoundWon,39.0


In [40]:
sub_group = clip_metrics_df.groupby('title').min()
game_sub = sub_group.loc[sub_group['game_name'] == "Demon's Souls"]

In [68]:
clip_metrics_df

,estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost,style,game_name,title,creator,view_count,likes_ratio
video_id,,,,,,,,,,,,
f0E8gLxw-II,306,511,6,1,0,0,m,Remnant: From the Ashes,His weakness is the Butt,TheRoundWon,51.0,0.857143
fuIgHicjm0k,233,871,34,2,3,0,m,Demon's Souls,Iconic,FeedMeACat,53.0,0.944444
1dSor9KrL34,229,927,8,2,0,0,m,Horizon Forbidden West,Enemies can do this now,TheRoundWon,10.0,0.800000
hFgJfliPTpA,175,596,17,-1,0,0,m,Remnant: From the Ashes,Remember to roll forward,TheRoundWon,61.0,1.062500
NFDigvjBh2A,165,782,16,5,0,0,m,Demon's Souls,Back to reality,TheRoundWon,10.0,0.761905
...,...,...,...,...,...,...,...,...,...,...,...,...
49WSCEmA7X4,0,1,0,0,0,0,m,Clash Royale,My must watch players,TheRoundWon,7.0,0.000000
cXdbITNtwPw,0,2,0,0,0,0,f,Horizon Forbidden West,Weird people on horses,TheRoundWon,7.0,0.000000
V1FdmoOpP3Q,0,1,0,0,0,0,m,Remnant: From the Ashes,Pessimism vs Realism? (or Optimism?),TheRoundWon,39.0,0.000000


In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([html.Label("Pick a Game to See Activity"),
    dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': value, 'value': value} for key, value in games.items()],
                value="Demon's Souls"
            ),

    html.Div([dcc.Graph(id='graph')], style={'display': 'inline-block', 'padding': '0 20'})], 
    style = {'display': 'inline-block',  'margin': 'auto' }),
    
    


]
)

@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input("crossfilter-xaxis-column", "value")]
)
def update_figure(value):
    fig = make_subplots(rows=1, cols=4, shared_yaxes=True, vertical_spacing=0.02)
    sub_group = clip_metrics_df.groupby('title').min()
    game_sub = sub_group.loc[sub_group['game_name'] == value]
    sub_filter = clip_metrics_df.loc[clip_metrics_df['game_name'] == value]
    mobile_sub = sub_filter.loc[sub_filter['style'] == 'm']
    full_sub = sub_filter.loc[sub_filter['style'] == 'f']
    # creators = mobile_sub['creator'].unique()
    for creator, value in creators.items():
        fig.add_trace(go.Bar(x=game_sub.loc[game_sub.creator == creator, 'view_count'].values, y=game_sub.loc[game_sub.creator == creator].index, orientation='h', name=creator, legendgroup=creator, marker={'color': value}), row=1, col=1)
        fig.add_trace(go.Bar(x=mobile_sub.loc[mobile_sub.creator == creator, 'views'].values, y=mobile_sub.loc[mobile_sub.creator == creator, 'title'].values, orientation='h', name='Shorts', legendgroup = creator, showlegend=False, marker={'color': 'blue'} ), row=1, col=2)
        fig.add_trace(go.Bar(x=full_sub.loc[full_sub.creator == creator, 'views'].values, y=full_sub.loc[full_sub.creator == creator, 'title'].values, orientation='h', name='Full Screen',  legendgroup = creator, showlegend=False, marker={'color': 'purple'} ), row=1, col=2)
        fig.add_trace(go.Scatter(x=mobile_sub.loc[mobile_sub.creator == creator,'likes_ratio'].values, y=mobile_sub.loc[mobile_sub.creator == creator,'title'].values, mode='markers', name="Shorts", legendgroup = creator, showlegend=False, marker={'color': 'blue', 'size': mobile_sub.loc[mobile_sub.creator == creator,'likes'].values}), row=1, col=3)
        fig.add_trace(go.Scatter(x=full_sub.loc[full_sub.creator == creator,'likes_ratio'].values, y=full_sub.loc[full_sub.creator == creator,'title'].values, mode='markers', name="Full Screen", legendgroup = creator, showlegend=False, marker={'color': 'purple', 'size': full_sub.loc[full_sub.creator == creator,'likes'].values}), row=1, col=3)
        
        fig.add_trace(go.Bar(x=sub_filter.loc[sub_filter.creator == creator, 'subscribersGained'].values, y=sub_filter.loc[sub_filter.creator == creator,'title'].values, orientation='h', name=creator, legendgroup = creator, showlegend=False, marker={'color': 'green'} ), row=1, col=4)
        fig.add_trace(go.Bar(x=sub_filter.loc[sub_filter.creator == creator, 'subscribersLost'].values, y=sub_filter.loc[sub_filter.creator == creator,'title'].values, orientation='h', name=creator, legendgroup = creator, showlegend=False, marker={'color': 'red'} ), row=1, col=4)
    fig.update_layout( width=1000, height=1000, barmode='stack', xaxis3 = {'range': [0,1], 'tickformat': ".0%"}, xaxis4={'dtick': 1} )
    return fig

    # html.Div([html.Div([html.H1("Like Monitoring by Name")],  {'width': '49%', 'display': 'inline-block', 'margin': '10px', 'float': 'right'}),
        #     html.H1("Like Monitoring by Name"),
        #     dcc.Dropdown(
        #         id='crossfilter-xaxis-column',
        #         options=[{'label': value, 'value': value} for key, value in games.items()],
        #         value="Demon's Souls"
        #     )
        # ], style = {'width': '49%', 'display': 'inline-block', 'margin': '10px', 'float': 'right'})
        # # ])
        # # ])

app.run_server(mode='inline')

In [ ]:
def completeThumbnails(playlist, default_mode = 'default'):
    default = playlist['snippet']['thumbnails'][default_mode]
    checks = ['default', 'medium', 'high', 'standard']
    for chck in checks:
        if not bool(playlist['snippet']['thumbnails'].get(chck)):
            playlist['snippet']['thumbnails'][chck] = default
    return playlist

In [ ]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None

mapped_as_full_counts = {}
mapped_full_vids = {}
with Session(engine) as session:
    for title in session.query(Clip_Tracker.title).where(Clip_Tracker.published == PublishingStatus.f).order_by(Clip_Tracker.view_count.desc()).all():
        mapped_full_vids[title[0]] = []
        mapped_as_full_counts[title[0]] = 0
shorts_pattern = f"[sS]hort" # check for YT short videos
with Session(engine) as session: # initialize session to sql using context manager
    for playlist in playlists['items']: # loop through the differen playlists
        playlist = completeThumbnails(playlist)
        for key, thumbnail in playlist['snippet']['thumbnails'].items(): # first ensure that thumbnails are mapped in thumbnail table
            if not bool(session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()):
                session.add(Thumbnails(asset=thumbnail['url'], width = thumbnail['width'], height=thumbnail['height']))
                session.commit()
            # map the thumbnail table ids to the playlist dictionary
            playlist['snippet']['thumbnails'][key]['id'] = session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()[0]

        # if the playlist id is not in the table, add it
        if not bool(session.query(PlayList_yt).where(PlayList_yt.id == playlist['id']).first()):
            session.add(PlayList_yt(id=playlist['id'], title=playlist['snippet']['title'], 
            tb_default = playlist['snippet']['thumbnails']['default']['id'], 
            tb_medium = playlist['snippet']['thumbnails']['medium']['id'],
            tb_high = playlist['snippet']['thumbnails']['high']['id'], 
            tb_standard = playlist['snippet']['thumbnails']['standard']['id']
            ))
            session.commit()

        # else, update the table with the current values if it's already in the table - this is to capture if thumbnails or titles have changed
        else:
            session.execute(
                update(PlayList_yt).where(PlayList_yt.id == playlist['id']).values(dict(title=playlist['snippet']['title'], 
            tb_default = playlist['snippet']['thumbnails']['default']['id'], 
            tb_medium = playlist['snippet']['thumbnails']['medium']['id'],
            tb_high = playlist['snippet']['thumbnails']['high']['id'], 
            tb_standard = playlist['snippet']['thumbnails']['standard']['id']))
            )
            session.commit()
        if "Shorts" in playlist['snippet']['title']:
            style = VideoStyle.m
        else:
            style = VideoStyle.f
        # next find all the videos in each playlist
        playlist_Items = service.playlistItems().list(part="id,snippet", playlistId=playlist['id']).execute()
            
        nextPage = playlist_Items['nextPageToken']
        while nextPage != None:
            seed_list = service.playlistItems().list(part="id,snippet", playlistId=playlist['id'], pageToken = nextPage).execute()
            nextPage = seed_list['nextPageToken']
            playlist_Items['items'] = playlist_Items['items'] + seed_list['items']
        for video in playlist_Items['items']:
            try:
                video = completeThumbnails(video)
                title = video['snippet']['title']
                description = video['snippet']['description']
                    # check whether the title or description have the word "short" in it
                # combine the tags into a  single string due to datatype limitations in database - leave as array if your database can store arrays
                tags = " | ".join(service.videos().list(part='snippet,contentDetails', id=video['snippet']['resourceId']['videoId']).execute()['items'][0]['snippet']['tags'])
                

                # to capture titles with title conventon used in yt_videoupload.py
                title_components = title.split("|")
                if len(title_components) > 1:
                    titleName = title_components[-1].strip()
                
                else: # incase title does not follow current convention
                    titleName = title
                
                try:
                    mapped_as_full_counts[titleName] += 1
                    mapped_full_vids[titleName].append(style)
                except:
                    pass

            
                # find clip and original filename in clips database
                for clip_id, filename in session.query(Clip_Tracker.id, Clip_Tracker.video_name).where(Clip_Tracker.title == titleName).all():

                    if not bool(clip_id):
                        print(title)
                    else:
                        # loop through the thumbnail images to ensure they are in the thumbnail database
                        for key, thumbnail in video['snippet']['thumbnails'].items():
                            if not bool(session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()):
                                session.add(Thumbnails(asset=thumbnail['url'], width = thumbnail['width'], height=thumbnail['height']))
                                session.commit()
                            video['snippet']['thumbnails'][key]['id'] = session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()[0]
                        # VideoID is not in the Video_yt table - add it!    
                        if not bool(session.query(Video_yt).where(Video_yt.id == video['snippet']['resourceId']['videoId']).first()):
                            session.add( Video_yt(id = video['snippet']['resourceId']['videoId'], 
                            clip_id=clip_id, 
                            title = title,
                            filename = filename,
                            style = style,
                            description = description,
                            tb_default = video['snippet']['thumbnails']['default']['id'], 
                            tb_medium = video['snippet']['thumbnails']['medium']['id'],
                            tb_high = video['snippet']['thumbnails']['high']['id'], 
                            tb_standard = video['snippet']['thumbnails']['standard']['id'],
                            tags = tags
                            
                            )
                            )
                            session.commit()
                        # If it's already in there, update our record of current descriptiption / thumbnails.
                        else:
                            if style == VideoStyle.m:
                                print(video)
                            session.execute(
                                update(Video_yt).where(Video_yt.id == video['snippet']['resourceId']['videoId']).values(
                                    dict(
                                        clip_id=clip_id, 
                                        title = title,
                                        filename = filename,
                                        style = style,
                                        description = description,
                                        tb_default = video['snippet']['thumbnails']['default']['id'], 
                                        tb_medium = video['snippet']['thumbnails']['medium']['id'],
                                        tb_high = video['snippet']['thumbnails']['high']['id'], 
                                        tb_standard = video['snippet']['thumbnails']['standard']['id'],
                                        tags = tags
                                    )
                                )
                                )
                            session.commit()
                        if not bool(session.query(yt_vid_pl_mapper).where(yt_vid_pl_mapper.c.yt_video_id == video['snippet']['resourceId']['videoId']).where(yt_vid_pl_mapper.c.yt_playlist_id ==playlist['id']).first()):
                            session.execute(insert(yt_vid_pl_mapper).values({"yt_playlist_id": playlist['id'], 'yt_video_id': video['snippet']['resourceId']['videoId']}))
            except Exception as e:
                print("failed", e)

In [ ]:
playlist_Items['items'][0]['snippet']

In [ ]:
with Session(engine)as session:
    for clip_id, game_title, clip_creator, console, clip_title, video_name, clip_url, publishing_status in session.query(Clip_Tracker.id, Game_Meta.game_name, Clip_Tracker.creator_name, Game_Meta.platform, Clip_Tracker.title, Clip_Tracker.video_name, Clip_Tracker.url, Clip_Tracker.published).select_from(join(Game_Meta,Clip_Tracker)).where(and_(Clip_Tracker.published != PublishingStatus.f,  Clip_Tracker.published != None, Clip_Tracker.mobiles_videos_processed == True)).order_by(Clip_Tracker.view_count.desc()).all():
        print(clip_id)

In [ ]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None
all_items = {}
for playlist in playlists['items']: 
    playlist_Items = service.playlistItems().list(part="id,snippet", playlistId=playlist['id']).execute()
    all_items[playlist['id']] = {'playlist_items': None, "videos": []}
    all_items[playlist['id']]['playlist_items'] = playlist_Items
    if bool(playlist_Items.get('nextPageToken')):
        nextPage = playlist_Items['nextPageToken']
    else:
        nextPage = None
    while nextPage != None:
        seed_list = service.playlistItems().list(part="id,snippet", playlistId=playlist['id'], pageToken = nextPage).execute()
        if bool(seed_list.get('nextPageToken')):
            nextPage = seed_list['nextPageToken']
        else:
            nextPage = None
        playlist_Items['items'] = playlist_Items['items'] + seed_list['items']
        for video in playlist_Items['items']:
            all_items[playlist['id']]['videos'].append(service.videos().list(part='snippet,contentDetails', id=video['snippet']['resourceId']['videoId']).execute())

In [ ]:
for key, value in all_items.items():
    for value2 in value['videos']:
        if bool(value2['items']):
            if value2['items'][0]['kind'] != 'youtube#video':
                # if bool()
                print(value2)

In [ ]:
value2['items']

In [ ]:
with Session(engine) as session:
    print(session.query(Video_yt.title, Video_yt.))

In [ ]:
playlist_Items['items']

In [ ]:
with Session(engine) as session:
    for clip_id in session.query(Clip_Tracker.id).where(Clip_Tracker.published == PublishingStatus.f).all():
        rows = session.query(Video_yt.title, Video_yt.style).where(Video_yt.clip_id == clip_id[0]).all()
        if len(rows) < 2 and bool(rows):
            session.query(Clip_Tracker).where(Clip_Tracker.id == clip_id[0]).update(values = {'published': invers_mapper[rows[0][1]]})
            session.commit()

            print(rows[0])

In [ ]:
rows[0]